# DepthAnything V2 深度推定API - Google Colab設定

このノートブックを使って、Google Colab上で深度推定APIサーバーを起動します。

## 1. 必要なライブラリのインストール

In [ ]:
# GPU使用可能か確認
!nvidia-smi

In [ ]:
# 必要なライブラリをインストール
!pip install torch torchvision transformers
!pip install gradio
!pip install pyngrok
!pip install opencv-python-headless
!pip install Pillow numpy

## 2. ngrokトークンの設定

1. [ngrok.com](https://ngrok.com)でアカウント作成
2. Dashboard > Your Authtoken からトークンをコピー
3. 下のセルでトークンを設定

In [ ]:
# ngrokトークンを設定
import pyngrok
from pyngrok import ngrok

# ここにngrokトークンを入力
NGROK_TOKEN = "YOUR_NGROK_TOKEN_HERE"  # 実際のトークンに置き換え

ngrok.set_auth_token(NGROK_TOKEN)
print("ngrok認証完了")

## 3. 深度推定APIコードの準備

In [ ]:
# APIコードを作成
api_code = '''
import os
import io
import base64
import numpy as np
from PIL import Image
import torch
import gradio as gr
from transformers import pipeline, AutoImageProcessor, AutoModelForDepthEstimation
import cv2
from pyngrok import ngrok

class DepthEstimationAPI:
    def __init__(self):
        self.device = "cuda" if torch.cuda.is_available() else "cpu"
        print(f"Using device: {self.device}")
        
        # DepthAnything V2モデルの初期化
        self.model_name = "depth-anything/Depth-Anything-V2-Small-hf"
        print(f"Loading model: {self.model_name}")
        self.processor = AutoImageProcessor.from_pretrained(self.model_name)
        self.model = AutoModelForDepthEstimation.from_pretrained(self.model_name)
        self.model.to(self.device)
        
        print(f"Model loaded successfully on {self.device}")
    
    def estimate_depth(self, image):
        """深度推定のメイン処理"""
        try:
            if image is None:
                return None, None
            
            # RGBに変換
            if image.mode != \'RGB\':
                image = image.convert(\'RGB\')
            
            original_size = image.size
            
            # モデル推論
            inputs = self.processor(images=image, return_tensors="pt")
            inputs = {k: v.to(self.device) for k, v in inputs.items()}
            
            with torch.no_grad():
                outputs = self.model(**inputs)
                predicted_depth = outputs.predicted_depth
            
            # 深度マップの後処理
            depth = predicted_depth.squeeze().cpu().numpy()
            
            # 正規化 (0-255)
            depth_min = depth.min()
            depth_max = depth.max()
            depth_normalized = ((depth - depth_min) / (depth_max - depth_min) * 255).astype(np.uint8)
            
            # 元のサイズにリサイズ
            depth_resized = cv2.resize(depth_normalized, original_size, interpolation=cv2.INTER_LINEAR)
            
            # カラーマップを適用 (viridis)
            depth_colored = cv2.applyColorMap(depth_resized, cv2.COLORMAP_VIRIDIS)
            depth_colored = cv2.cvtColor(depth_colored, cv2.COLOR_BGR2RGB)
            
            # PILイメージに変換
            depth_image = Image.fromarray(depth_colored)
            
            return image, depth_image
            
        except Exception as e:
            print(f"Error in depth estimation: {e}")
            return None, None

# グローバルAPI インスタンス
print("Initializing DepthEstimation API...")
api = DepthEstimationAPI()

def gradio_interface():
    """Gradio インターフェースの設定"""
    
    interface = gr.Interface(
        fn=api.estimate_depth,
        inputs=gr.Image(type="pil", label="入力画像"),
        outputs=[
            gr.Image(type="pil", label="元画像"),
            gr.Image(type="pil", label="深度マップ")
        ],
        title="DepthAnything V2 深度推定API",
        description="DepthAnything V2モデルを使用した高精度深度推定",
        allow_flagging="never"
    )
    
    return interface

# Gradio インターフェースを作成
print("Setting up Gradio interface...")
demo = gradio_interface()

# ngrokでパブリックURLを取得
print("Setting up ngrok tunnel...")
public_url = ngrok.connect(7860)
print(f"\n" + "=" * 60)
print(f"🚀 API Server is running!")
print(f"Public URL: {public_url}")
print(f"\n📋 Frontend環境変数に設定してください:")
print(f"NEXT_PUBLIC_BACKEND_URL={public_url}")
print(f"=" * 60)

# サーバー起動
demo.launch(
    server_name="0.0.0.0",
    server_port=7860,
    share=False,
    debug=True
)
'''

# ファイルに保存
with open('depth_api.py', 'w', encoding='utf-8') as f:
    f.write(api_code)

print("APIコード準備完了")

## 4. APIサーバー起動

このセルを実行するとAPIサーバーが起動し、パブリックURLが表示されます。

In [ ]:
# APIサーバーを起動
exec(open('depth_api.py').read())

## 5. フロントエンド設定

上で表示されたURLを、フロントエンドの環境変数に設定してください。

```bash
# frontend/.env.local
NEXT_PUBLIC_BACKEND_URL=https://xxxxxxxx.ngrok.io
```

## 使用上の注意

1. **Colab セッション**: Colabは12時間でセッションが切れます
2. **ngrok URL**: 無料版は8時間で期限切れになります
3. **GPU使用量**: T4 GPUを効率的に使用します
4. **メモリ使用量**: 約2-3GB程度使用します

## トラブルシューティング

- モデル読み込みエラー: GPUメモリ不足の可能性があります
- ngrok接続エラー: トークンの設定を確認してください
- タイムアウトエラー: ランタイムを再起動してください